## How we can connect with the datasource
We can utilize this as well but it's fine to just use normal spark streaming, so no structured spark streaming

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
socketDF = spark.readStream.format("socket").option("host", "seppe.net").option("port", 7778).load()

socketDF.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import from_json, schema_of_json

In [6]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()

In [7]:
query = socketDF.writeStream.trigger(processingTime='2 seconds').foreachBatch(process_row).start()  

0
1
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|             comment|label|        name_user|            text_new|            text_old|          title_page|            url_page|
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|replaced: long di...| safe|Chris the speller|{{Infobox company...|{{Infobox company...|Frontier Communic...|//en.wikipedia.or...|
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+

2
+--------------------+-----+-------------------+--------------------+--------------------+----------+--------------------+
|             comment|label|          name_user|            text_new|            text_old|title_page|            url_page|
+--------------------+-----+-------------------+--------------------+--------------------+--

In [8]:
query.stop()

4
+------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|     comment|label| name_user|            text_new|            text_old|          title_page|            url_page|
+------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|→‎Early life| safe|Nedrutland|{{Refimprove|date...|{{Refimprove|date...|William Stephen R...|//en.wikipedia.or...|
+------------+-----+----------+--------------------+--------------------+--------------------+--------------------+

